In [16]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import sklearn

In [82]:
# Only use the data generated during preparation process to classify
status = 1 
df = pd.read_csv('data/Processed/EMG/SS2023_FukuS_Session1_Shimmer_F16C_Calibrated_SD.csv') #ステータスが1になるデータを取得

window_percentage = 1

window_length = []
for i in range(1, 52):
    window_length.append(len(df[df.order == i]))

preparation_window_length = []
for i in range(1, 52):
    action = df[df.order == i]
    preparation_window_length.append(len(action[action.status == 1]))

action_end_point = []
preparation_end_point = []
accu_1 = 0
accu_2 = 0
for i, length in enumerate(preparation_window_length):
    preparation_end_point.append(accu_1 + length)
    accu_1 += window_length[i] - 1
    action_end_point.append(accu_1)
    
selected_window_size = []
for length in preparation_window_length:
    selected_window_size.append(int(length * window_percentage))
selected_indexes = []

for i, size in enumerate(selected_window_size):
    selected_indexes.append(np.arange(preparation_end_point[i] - size + 1, preparation_end_point[i] + 1))
    print(preparation_end_point[i] - size + 1, preparation_end_point[i] + 1)
selected_indexes = np.concatenate(selected_indexes)

ch1 = np.array(df.CH1)[selected_indexes]
ch2 = np.array(df.CH2)[selected_indexes]
target = np.array(df.actions)[selected_indexes]
target = target.reshape(1, -1)

data = np.stack((ch1, ch2))

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data.T, target.T, test_size=0.3, random_state=0)


1 1537
3072 4608
6134 7670
9205 10741
12276 13812
15337 16873
18407 19942
21478 23014
24548 26084
27619 29155
30689 32225
33751 35288
36823 38359
39894 41430
42965 44501
46036 47572
49107 50642
52177 53713
55248 56784
58318 59854
61379 62915
64449 65984
67520 69056
70591 72127
73662 75198
76733 78269
79804 81340
82875 84410
85935 87471
89006 90542
92077 93612
95147 96683
98218 99753
101288 102824
104359 105895
107429 108965
110500 112036
113571 115108
116643 118179
119705 121241
122776 124312
125847 127382
128917 130453
131987 133523
135048 136584
138119 139654
141189 142612
144147 145684
147218 148754
150289 151825
153360 154896


In [83]:
from sklearn.model_selection import cross_val_score, KFold
from scipy.stats import sem

from sklearn.svm import SVC
svc_1 = SVC(kernel='linear')

def evaluate_cross_validation(clf, x, y, K):                        
    cv = KFold(n_splits=K, random_state=0,shuffle=True)
    accuracy = cross_val_score(clf,x,y,cv=cv, scoring='accuracy')
    print(accuracy)
    print ("Mean accuracy: {} (+/-{})".format( np.mean (accuracy), sem(accuracy)))

evaluate_cross_validation(svc_1, data.T, target.T , 5)

/Users/celia/virtualenvs/jupyter/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/celia/virtualenvs/jupyter/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/celia/virtualenvs/jupyter/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/celia/virtualenvs/jupyter/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed w

[0.37324214 0.36499616 0.3722833  0.37710158 0.37895544]
Mean accuracy: 0.3733157256075468 (+/-0.002412731923009958)


In [81]:
from sklearn.model_selection import cross_val_score, KFold
from scipy.stats import sem

from sklearn.svm import SVC
svc_1 = SVC(kernel='linear')

def evaluate_cross_validation(clf, x, y, K):                        
    cv = KFold(n_splits=K, random_state=0,shuffle=True)
    accuracy = cross_val_score(clf,x,y,cv=cv, scoring='accuracy')
    print(accuracy)
    print ("Mean accuracy: {} (+/-{})".format( np.mean (accuracy), sem(accuracy)))

evaluate_cross_validation(svc_1, data.T, target.T , 5)

/Users/celia/virtualenvs/jupyter/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/celia/virtualenvs/jupyter/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/celia/virtualenvs/jupyter/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/celia/virtualenvs/jupyter/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed w

[0.38115331 0.38537271 0.37271449 0.37352941 0.37723785]
Mean accuracy: 0.3780015539512448 (+/-0.0023749096960632163)


In [79]:
from sklearn.model_selection import cross_val_score, KFold
from scipy.stats import sem

from sklearn.svm import SVC
svc_1 = SVC(kernel='linear')

def evaluate_cross_validation(clf, x, y, K):                        
    cv = KFold(n_splits=K, random_state=0,shuffle=True)
    accuracy = cross_val_score(clf,x,y,cv=cv, scoring='accuracy')
    print(accuracy)
    print ("Mean accuracy: {} (+/-{})".format( np.mean (accuracy), sem(accuracy)))

evaluate_cross_validation(svc_1, data.T, target.T , 5)

/Users/celia/virtualenvs/jupyter/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/celia/virtualenvs/jupyter/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/celia/virtualenvs/jupyter/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/celia/virtualenvs/jupyter/lib/python3.8/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed w

[0.37003841 0.368758   0.37878788 0.37089202 0.37673426]
Mean accuracy: 0.3730421141382105 (+/-0.001982983693179724)


In [88]:
len(np.argwhere(target == 2)) / len(target.T)

0.2940883172671252